In [8]:
import cv2
import mediapipe as mp
import numpy as np
from matplotlib import pyplot as plt
import os

Para conseguir criar o seu dataset com as imagens da mao de vcs vai precisar criar as pastas das letras, palavras ou numeros dentro da pasta Data, e dentro dela é necessario criar uma outra pasta para juntar as imagens da sua mao.


Seguindo a estrutura criada é so tirar as fotos e colocar dentro da pasta que vc criou

In [28]:
def capture_hand_image():
    # simbolo que vai tirar a foto, precisa alterar toda vez que for fazer a captura
    simbolo = '5'
    # numero da sua pasta, basta alterar uma unica vez
    num = 1

    folder_path = os.path.abspath(f'./Data/Fold{num}/{simbolo}')
    
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    counter = 0

    # Inicializa a câmera
    cap = cv2.VideoCapture(0)

    # Configuração da mediapipe
    with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=1,
        min_detection_confidence=0.7
    ) as hands:
        while cap.isOpened():
            ret, frame = cap.read()

            # Convertendo o frame para RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detecta as mãos no frame
            results = hands.process(image)

            # Desenha as landmarks da mão no frame
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS
                    )

            # Exibe o frame
            cv2.imshow('Hand Detection', cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

            # Captura a imagem quando a tecla 's' for pressionada
            if cv2.waitKey(10) & 0xFF == ord('s'):
                
        
                # Salva a imagem da mão isolada
                cv2.imwrite(f"{folder_path}/{counter}.jpg", cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                counter += 1

        
            if(counter > 9) : break
        # Libera os recursos
        cap.release()
        cv2.destroyAllWindows()


In [29]:
if __name__ == "__main__":
    capture_hand_image()